This Jupyter nootebook is essentially based on functions of the <b>Python Library MNE (https://mne.tools/stable/index.html).</b>

It contains the basic steps for processing EEG data:
* Loading
* Specify montage
* Re-referencing
* Filtering
* Data quality assessment #1: Filtered data
* Event annotations
* Artefact cleaning using Artefact Subspace Reconstructionb (ASR)
* Segmentation / Epoching
* Baseline correction
* Data quality assessment #2: Segment / Epoch
* Time frequency analyses (TFA)
* Saving the results

### ____________________________________________________________________________
### Loading the necessary libraries 

#### EEG specific libraries

In [401]:
import mne                                             # all the stuff for artefact cleaning, epoching, ERP and TFA computation

#### Libraries for data handling, data display, mathematical computations and file handling

In [402]:
import os                         # operating system related functions
import ipyfilechooser             # interactive file chooser
import numpy as np                # basic mathematical functions
import pandas as pd               # excel-like table functions
import seaborn as sns             # data plotting
import matplotlib.pyplot as plt   # again, data plotting
%matplotlib qt                    
import warnings                   # only to suppress annoying warning messages 
mne.set_log_level("ERROR")        # show only errors (no warnings or information messages)
warnings.simplefilter(action='ignore')

In [403]:
# specific frequency bands
frequencyBands = {"delta": [0.5, 4.0],
                  "theta": [4.0, 8.0],
                  "alpha": [8.0, 11.0],
                  "sigma": [11.0, 15.0],
                  "beta":  [15.0, 30.0],
                  "gamma": [30.0, 35.0]}

# rejection criteria
rejectionCriteria = dict(eeg = 1.5e-4) # 100 µV

# default display scaling, use e-6 for display in microvolts
scal = {'eeg': 1e-4}  

# cap design
easycap_montage = mne.channels.make_standard_montage("easycap-M1")

#### 1) Loading data ans set montage

In [404]:
data_folder = "/Volumes/SanDisk/data"
exp_id = "010_syl_4m_bil_20241022"
# load eeg data
path = os.path.join(data_folder, exp_id, "eeg", f"{exp_id}.vhdr")
if os.path.exists(path):
    raw = mne.io.read_raw_brainvision(path, preload = True)
else:
    raw = None
    print(f"EEG data not found in {path}")
    
# create folder for analysis output
analysis_path = os.path.join(data_folder, exp_id, "eeg", "analysis")
os.makedirs(analysis_path, exist_ok=True)

In [405]:
events, event_id = mne.events_from_annotations(raw)
print(f"Overall number of triggers (including stimtrak): {len(events)}") 
print(f"Number of computer triggers: {len([e for e in events if e[2] != 10001 and  e[2] != 99999 ])} (400 is expected for full experiment)") 

Overall number of triggers (including stimtrak): 1002
Number of computer triggers: 400 (400 is expected for full experiment)


#### 2) Specify montage<br>3) Re-referencing<br>4) Filtering<br>5) Data quality assessment #1: Filtered data

In [406]:
# set used montage
raw.set_montage(easycap_montage,on_missing='ignore')
# # re-referencing to mean
raw_AvgRef = raw.copy()
raw_AvgRef.set_eeg_reference("average")
# band-pass filter
raw_AvgRef_filt = raw_AvgRef.copy().filter(l_freq=0.5, h_freq = 35)
raw_AvgRef_filt.plot_psd()
raw_AvgRef_filt.info

<Info | 9 non-empty values
 bads: []
 ch_names: C3, F3, Fz, F4, C4
 chs: 5 EEG
 custom_ref_applied: True
 dig: 8 items (3 Cardinal, 5 EEG)
 highpass: 0.5 Hz
 lowpass: 35.0 Hz
 meas_date: 2024-10-22 09:25:23 UTC
 nchan: 5
 projs: []
 sfreq: 1000.0 Hz
>

In [407]:
# Discard stimtrak triggers that do not follow immediately after a computer trigger

descriptions = []
onsets = []
durations = []
orig_times = []

last_desc = ""
last_onset = ""
for annotation in raw_AvgRef_filt.annotations:
    if "Push/P" in annotation["description"] and "Stimulus/S" in last_desc:
        if annotation["onset"] - last_onset <= 0.5:
            descriptions.append(last_desc)
            onsets.append(annotation["onset"])
            durations.append(annotation["duration"])
            orig_times.append(annotation["orig_time"])
        else:
            print(f"WARNING: Stimtrak trigger too late at time {annotation['onset']} -- computer trigger at {last_onset}. Skipping trigger.")

    if "Stimulus/S" in annotation["description"] and "Stimulus/S" in last_desc:
        print(f"WARNING: Missing stimtrak trigger at time {last_onset}.")
   
    
    last_desc = annotation["description"]
    last_onset = annotation["onset"]
    
raw_AvgRef_filt_select = raw_AvgRef_filt.copy()
raw_AvgRef_filt_select.set_annotations(mne.Annotations(onset=onsets, duration=durations, description=descriptions))   
print(f"\nNumber of triggers left after adjustments: {len(raw_AvgRef_filt_select.annotations)}") 


Number of triggers left after adjustments: 399


In [408]:
from mne import Annotations 
from collections import defaultdict


descriptions = []
onsets = []
durations = []
orig_times = []

last_deviant = True
last_onset = 0

a = raw_AvgRef_filt_select.annotations[0]
descriptions.append(a["description"])
onsets.append(a["onset"])
durations.append(a["duration"])
orig_times.append(a["orig_time"])

for a in raw_AvgRef_filt_select.annotations[1:]:
    _, n = a["description"].replace("  "," ").split(" ")
    if n[-1] == "0":
        deviant = False
    else:
        deviant = True
    onset = a["onset"]
    if not last_deviant and (onset-last_onset) < 10:
        descriptions.append(a["description"])
        onsets.append(a["onset"])
        durations.append(a["duration"])
        orig_times.append(a["orig_time"])
    last_deviant = deviant
    last_onset = onset




raw_AvgRef_filt_new = raw_AvgRef_filt_select.copy()
raw_AvgRef_filt_new.set_annotations(Annotations(onset=onsets, duration=durations, description=descriptions))
print(f"{len(raw_AvgRef_filt_new.annotations)} triggers left out of {len(raw_AvgRef_filt_select.annotations)} after removing first triggers and post-deviant triggers.")


303 triggers left out of 399 after removing first triggers and post-deviant triggers.


In [409]:
standards = [0,0,0,0]
deviants = [0,0,0,0]
conditions = ['SSpec','SDur','NSSpec','NSDur']
for ii in raw_AvgRef_filt_new.annotations:
    if "Stimulus" in ii['description']:
        if ii['description'][-2] == '1': 
            if ii['description'][-1] == '0':
                standards[0] = standards[0] + 1
            else:
                deviants[0] = deviants[0] + 1
        if ii['description'][-2] == '2': 
            if ii['description'][-1] == '0':
                standards[1] = standards[1] + 1
            else:
                deviants[1] = deviants[1] + 1
        if ii['description'][-2] == '3': 
            if ii['description'][-1] == '0':
                standards[2] = standards[2] + 1
            else:
                deviants[2] = deviants[2] + 1
        if ii['description'][-2] == '4': 
            if ii['description'][-1] == '0':
                standards[3] = standards[3] + 1
            else:
                deviants[3] = deviants[3] + 1
            
for ii in range(4):
    print(f'{conditions[ii]}: #standards = {standards[ii]} | #deviants = {deviants[ii]}')            

SSpec: #standards = 56 | #deviants = 20
SDur: #standards = 56 | #deviants = 20
NSSpec: #standards = 55 | #deviants = 20
NSDur: #standards = 56 | #deviants = 20


#### 6) Artefact correction with Arefact Subspace Reconstruction (ASR)

In [410]:
from asrpy import ASR

raw_AvgRef_filt_ASRcorrected = raw_AvgRef_filt_new.copy()
asr = ASR(sfreq=raw_AvgRef_filt_ASRcorrected.info["sfreq"], cutoff=15)
asr.fit(raw_AvgRef_filt_ASRcorrected)
raw_AvgRef_filt_ASRcorrected = asr.transform(raw_AvgRef_filt_ASRcorrected)
raw_AvgRef_filt_ASRcorrected.plot(scalings = scal)

#### 7) Epoching

In [411]:
evt = mne.events_from_annotations(raw_AvgRef_filt_ASRcorrected, event_id = {'Stimulus/S 10': 1001, 
                                                                            'Stimulus/S 11': 1002,
                                                                            'Stimulus/S 20': 1003,
                                                                            'Stimulus/S 21': 1004,
                                                                            'Stimulus/S 30': 1005, 
                                                                            'Stimulus/S 31': 1006,
                                                                            'Stimulus/S 40': 1007,
                                                                            'Stimulus/S 41': 1008})
epochs = mne.Epochs(raw_AvgRef_filt_ASRcorrected, 
                    events = evt[0],
                    event_id = {'SSpec(S)':1001, 'SSpec(D)' : 1002, 
                                'SDur(S)':1003,  'SDur(D)' : 1004,
                                'NSSpec(S)':1005, 'NSSpec(D)' : 1006, 
                                'NSDur(S)':1007,  'NSDur(D)' : 1008},
                    baseline=(-0.05,0.05), 
                    tmin = -0.1, 
                    tmax = 0.6, 
                    reject_tmin = 0.0, 
                    reject_tmax = 0.6,
                    reject = {'eeg': 1e-4},
                    flat = {'eeg': 1e-10},
                    detrend = 1,
                    preload = True)
epochs.plot(scalings = scal)
print(f'Good epochs = {len(epochs)}')
excluded_epochs_auto = [i + 1 for i, reason in enumerate(epochs.drop_log) if reason]
print(f"Epoch numbers, which were automatically excluded: {excluded_epochs_auto}")

Good epochs = 303
Epoch numbers, which were automatically excluded: []


In [412]:
excluded_epochs_man = [i for i, reason in enumerate(epochs.drop_log) if reason and  "USER" in reason]
notes = os.path.join(analysis_path, f"notes.txt")
with open(notes, "w") as f:
    f.write(f"Channels which were excluded by manual inspection: {epochs.info['bads']}\n")
    f.write(f"Epoch numbers which were excluded by manual inspection: {excluded_epochs_man}\n")
with open(notes, "r") as f:
    print(f.read())

Channels which were excluded by manual inspection: []
Epoch numbers which were excluded by manual inspection: []



In [413]:
evokedSSpecStandard = epochs['SSpec(S)'].average()
evokedSSpecStandard.save(os.path.join(analysis_path, f"{exp_id}_SSpec(S).erp"),overwrite=True)

evokedSSpecDeviant = epochs['SSpec(D)'].average()
evokedSSpecDeviant.save(os.path.join(analysis_path, f"{exp_id}_SSpec(D).erp"),overwrite=True)

evokedNSSpecStandard = epochs['NSSpec(S)'].average()
evokedNSSpecStandard.save(os.path.join(analysis_path, f"{exp_id}_NSSpec(S).erp"),overwrite=True)

evokedNSSpecDeviant = epochs['NSSpec(D)'].average()
evokedNSSpecDeviant.save(os.path.join(analysis_path, f"{exp_id}_NSSpec(D).erp"),overwrite=True)

evokedSDurStandard = epochs['SDur(S)'].average()
evokedSDurStandard.save(os.path.join(analysis_path, f"{exp_id}_SDur(S).erp"),overwrite=True)

evokedSDurDeviant = epochs['SDur(D)'].average()
evokedSDurDeviant.save(os.path.join(analysis_path, f"{exp_id}_SDur(D).erp"),overwrite=True)

evokedNSDurStandard = epochs['NSDur(S)'].average()
evokedNSDurStandard.save(os.path.join(analysis_path, f"{exp_id}_NSDur(S).erp"),overwrite=True)

evokedNSDurDeviant = epochs['NSDur(D)'].average()
evokedNSDurDeviant.save(os.path.join(analysis_path, f"{exp_id}_NSDur(D).erp"),overwrite=True)

mne.viz.plot_compare_evokeds([evokedSSpecStandard,evokedSSpecDeviant], picks=['F3'], combine = 'mean')
mne.viz.plot_compare_evokeds([evokedNSSpecStandard,evokedNSSpecDeviant], picks=['F3'], combine = 'mean')
mne.viz.plot_compare_evokeds([evokedSDurStandard,evokedSDurDeviant], picks=['F3'], combine = 'mean')
mne.viz.plot_compare_evokeds([evokedNSDurStandard,evokedNSDurDeviant], picks=['F3'], combine = 'mean')

fig = mne.viz.plot_compare_evokeds([evokedSSpecStandard,evokedSSpecDeviant], picks=['F3'], combine = 'mean')
fig[0].savefig(os.path.join(analysis_path, f"{exp_id}_SSpec.png"))
fig = mne.viz.plot_compare_evokeds([evokedNSSpecStandard,evokedNSSpecDeviant], picks=['F3'], combine = 'mean')
fig[0].savefig(os.path.join(analysis_path, f"{exp_id}_NSSpec.png"))
fig = mne.viz.plot_compare_evokeds([evokedSDurStandard,evokedSDurDeviant], picks=['F3'], combine = 'mean')
fig[0].savefig(os.path.join(analysis_path, f"{exp_id}_SDur.png"))
fig = mne.viz.plot_compare_evokeds([evokedNSDurStandard,evokedNSDurDeviant], picks=['F3'], combine = 'mean')
fig[0].savefig(os.path.join(analysis_path, f"{exp_id}_SSpec.png"))
